In [1]:
from tdmclient import ClientAsync
import cv2
from scipy.spatial import distance as dist 
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import math
%matplotlib qt

import matplotlib.patches as patches

import numpy as np
client = ClientAsync()
node = await client.wait_for_node()

await node.lock()

Node d03d4c82-c7d9-461b-9284-191eb97b1e15

### Constant

In [2]:

# FILTERING
THYMIO_MMS = 0.3183 # Thymio's unit to mm/s
D_BASELINE = 95 # mm
DELTA_T = 0.5 # s
SIZE_PLOT = 100
KIDNAP_THRESHOLD = 5

# MOTION CONTROL
SUCCESS_THRESHOLD = 40 
VELOCITY_THRESHOLD = 75 

# VISION
SCALING_FACTOR = 20/3

# color filters
HSV_RED_MIN = np.array([165, 30, 30])
HSV_RED_MAX = np.array([180, 255, 255])

HSV_ORANGE_MIN = np.array([0, 30, 30])
HSV_ORANGE_MAX = np.array([15, 255, 255])

HSV_GREEN_MIN = np.array([65, 40, 40])
HSV_GREEN_MAX = np.array([85, 255, 255])

HSV_BLUE_MIN = np.array([85, 100, 100])
HSV_BLUE_MAX = np.array([110, 255, 255])

TOLERANCE_HSV = np.array([8, 100, 100])

# change those values with the camera 
RED_BGR = np.array([115, 95, 194], dtype = np.uint8)
BLUE_BGR = np.array([134, 73, 12], dtype= np.uint8)
GREEN_BGR = np.array([29, 72, 56], dtype = np.uint8)
ORANGE_BGR = np.array([134, 73, 12], dtype = np.uint8)

# camera and map parameters
CAMERA_WIDTH = 1024
CAMERA_HEIGHT = 768
BIRD_WIDTH = 660
BIRD_HEIGHT = 492
MAP_WIDTH = 330
MAP_HEIGHT = 246
REAL_WIDTH = 1100
REAL_HEIGHT = 820
CAMERA_CORNERS = [[0, 0], [CAMERA_WIDTH, 0], [CAMERA_WIDTH, CAMERA_HEIGHT], [0, CAMERA_HEIGHT]]
BIRD_CORNERS = [[0, 0], [BIRD_WIDTH, 0], [BIRD_WIDTH, BIRD_HEIGHT], [0, BIRD_HEIGHT]]
MAP_CORNERS = np.array([[0, 0], [MAP_WIDTH, 0], [MAP_WIDTH, MAP_HEIGHT], [0, MAP_HEIGHT]], dtype=np.float32)

# matching thresholds
RES_THRESHOLD_BLACK = 0.82
RES_THRESHOLD_RED = 0.4
RES_THRESHOLD_GREEN = 0.3

# binary thresholds
BINARY_THRESHOLD_BLACK = 30
BINARY_THRESHOLD_RED = 25
BINARY_THRESHOLD_GREEN = 50
BINARY_THRESHOLD_BLUE = 27

OVERLAP_THRESHOLD = 0
THYMIO_WIDTH = 17
TEMP_CENTER_Y = 53
TEMP_CENTER_X = 69

OBSTACLE_MAP_VALUE = -1
BACKGROUND_MAP_VALUE = 0

In [3]:
#templates
template_cross = cv2.imread('/Users/tjga9/Documents/Tomas/EPFL/MA1/Basic of Mobile Robotics/Control your Thymio in Python/Control your Thymio in Python/template_cross_dll.jpg')

method = eval('cv2.TM_CCORR_NORMED')

### Plotting Functions

In [4]:
def initialize_plot():
    fig,ax = plt.subplots()
    line, = ax.plot([], [], '--', label="Position with odometry", color ="black")
    ax.set_xlim(-SIZE_PLOT, SIZE_PLOT+100)
    ax.set_ylim(-SIZE_PLOT, SIZE_PLOT)
    ax.set_title("Real time Kalmann plot")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()
    ax.minorticks_on()
    ax.grid(which='major', color="0.75", linestyle='-')
    #ax.grid(which='minor', color="0.75", linestyle='--')
    plt.rcParams.update({
        "text.usetex": False,
        "font.family": "sans-serif"
    })

    return fig, ax, line

In [5]:
def display_map(map_grid, path, start, goal):
    cmap = ListedColormap(['white', 'black', 'blue', 'green', 'red', 'grey'])
    map_display = np.zeros_like(map_grid, dtype=object)

    # Assign colors based on the map grid values
    map_display[map_grid == -1] = 'black'  # Obstacles
    map_display[map_grid == 0] = 'white'   # Free space

    # for position in explored:
    #     if map_display[tuple(position)] == 'white':
    #         map_display[tuple(position)] = 'grey'  # Explored cells

    # Visualize the path
    for position in path:
        if map_display[position[0], position[1]] in ['white', 'grey']:
            map_display[position[0], position[1]] = 'blue'  # Path

    # map_display[5, 3] = 'yellow' # Weighted cell
    map_display[start[0], start[1]] = 'green'  # Start
    map_display[goal[0], goal[1]] = 'red'      # Goal

    # Convert color names to numbers for plotting
    color_mapping = {'white': 0, 'black': 1, 'blue': 2, 'green': 3, 'red': 4, 'grey': 5}
    map_numeric_display = np.vectorize(color_mapping.get)(map_display)
    fig, ax = plt.subplots(figsize=(6, 7))
    ax.imshow(map_numeric_display, cmap=cmap)
    ax.set_xticks(np.arange(-0.5, map_grid.shape[1], 1), minor=True)
    ax.set_yticks(np.arange(-0.5, map_grid.shape[0], 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)
    ax.tick_params(which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    plt.show()

### General Functions

In [6]:
def thymio_transform_pose(pose, start): # from (x, y)_v to (x, y)_t 
    start_x, start_y = start
 
    new_x = (pose[1] - start_y)
    new_y = -(pose[0] - start_x)
    
    return new_x, new_y

In [ ]:
def normalize_angle(angle):  # -π to π
    normalized_angle = (angle + np.pi) % (2 * np.pi) - np.pi
    return normalized_angle

In [8]:
print(np.rad2deg(normalize_angle(np.deg2rad(445))))

85.0


In [9]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

### Vision Functions

In [10]:
old_corners = CAMERA_CORNERS #variable that will be updated each time
is_cam_obstructed = False
map = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
bird_image = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
hsv_img = np.zeros((BIRD_HEIGHT, BIRD_WIDTH, 3))

In [11]:
def band_pass_hsv(bgr_color):
	"""given a color in bgr colorspace, returns the min and max values for the bandpass filter"""
	hsv_color = cv2.cvtColor(bgr_color.reshape(1, 1, 3), cv2.COLOR_BGR2HSV)[0][0]
	hsv_min = hsv_color - TOLERANCE_HSV
	hsv_max = hsv_color + TOLERANCE_HSV

	return hsv_min, hsv_max

In [12]:
def filter_hsv_to_binary_old(hsv_img, hsv_min, hsv_max, binary_threshold):
	"""transform a HSV image to a binary image with threshold of the desired color
	| hsv_img : image in the hsv colorspace
	| hsv_min, hsv_max : min and max of color filter
	| binary_threshold : threshold under which the pixel is 0 and over which the pixel is 255"""
	#keep only the desired color in hsv
	mask = cv2.inRange(hsv_img, hsv_min, hsv_max)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)

	grayscale = output[:, :, 2]
	#convert to binary
	threshold = binary_threshold
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
	return bw

In [13]:
def filter_to_hsv(image):
	"""Filters the image from BGR colorspace to HSV colorspace"""
	filtered_img = cv2.GaussianBlur(image,(5,5),1)
    
    # Convert BGR to HSV
	hsv_img = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2HSV)
	return hsv_img

In [14]:
def filter_hsv_to_binary(hsv_img, bgr_color, binary_threshold):
	"""filters the color of an hsv image to a binary image
	| hsv img : image in the hsv colorspace
	| bgr_color : color to filter in the bgr colorspace
	| binary threshold : value btw 0 and 255 for binary filter"""
	hsv_min, hsv_max = band_pass_hsv(bgr_color)
	mask = cv2.inRange(hsv_img, hsv_min, hsv_max)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)
	grayscale = output[:, :, 2]
	#convert to binary
	threshold = binary_threshold
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
	return bw

In [15]:
def filter_grayscale(img):
	"given image in bgr colorspace, convert it to grayscale then convert it to binary"
	#convert to grayscale both image and template
	img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	#convert to binary both image and template
	threshold = BINARY_THRESHOLD_BLACK
	ret, image_binary = cv2.threshold(img_grayscale, threshold, 255, cv2.THRESH_BINARY)

	return image_binary

In [16]:
def non_max_suppression_fast(boxes, overlapThresh):
	"""Makes sure to only have one set of coordinate per cross to find the corners of the map"""
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# if the bounding boxes integers, convert them to floats --
	# this is important since we'll be doing a bunch of divisions
	if boxes.dtype.kind == "i":
		boxes = boxes.astype("float")
	# initialize the list of picked indexes	
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list and add the
		# index value to the list of picked indexes
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		# find the largest (x, y) coordinates for the start of
		# the bounding box and the smallest (x, y) coordinates
		# for the end of the bounding box
		xx1 = np.maximum(x1[i], x1[idxs[:last]])
		yy1 = np.maximum(y1[i], y1[idxs[:last]])
		xx2 = np.minimum(x2[i], x2[idxs[:last]])
		yy2 = np.minimum(y2[i], y2[idxs[:last]])
		# compute the width and height of the bounding box
		w = np.maximum(0, xx2 - xx1 + 1)
		h = np.maximum(0, yy2 - yy1 + 1)
		# compute the ratio of overlap
		overlap = (w * h) / area[idxs[:last]]
		# delete all indexes from the index list that have
		idxs = np.delete(idxs, np.concatenate(([last],
			np.where(overlap > overlapThresh)[0])))
	# return only the bounding boxes that were picked using the
	# integer data type
	return boxes[pick].astype("int")

In [17]:
def order_points(pts):
	"""Orders points of rectangle so that they are in the order : top_left, top_right, bottom_right, bottom_left """
	# sort the points based on their x-coordinates
	xSorted = pts[np.argsort(pts[:, 0]), :]
	# grab the left-most and right-most points from the sorted
	# x-roodinate points
	leftMost = xSorted[:2, :]
	rightMost = xSorted[2:, :]
	# now, sort the left-most coordinates according to their
	# y-coordinates so we can grab the top-left and bottom-left
	# points, respectively
	leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
	(tl, bl) = leftMost
	# now that we have the top-left coordinate, use it as an
	# anchor to calculate the Euclidean distance between the
	# top-left and right-most points; by the Pythagorean
	# theorem, the point with the largest distance will be
	# our bottom-right point
	D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
	(br, tr) = rightMost[np.argsort(D)[::-1], :]
	# return the coordinates in top-left, top-right,
	# bottom-right, and bottom-left order
	return np.array([tl, tr, br, bl], dtype="float32")

In [18]:
def find_4_corners(img, template, method):
	"""Finds the coordiantes of the corners of the map; 	
	arguments :	
	| img : image in the bgr color space;	
	| template : image that has been filtered and is in the binary color space;	
	| method : cross-correlation method for template matching
	"""
	is_found = False
	
	#filter the image
	filtered_img = cv2.GaussianBlur(img,(5,5), 1)
	
	#convert image and template to binary
	image_binary = filter_grayscale(filtered_img)
	template_binary = filter_grayscale(template)
	
	#cv2.imshow("binary of black", image_binary)
	#get template dimensions
	c, w, h  = template.shape[::-1]
	#print(template.shape[::-1])
	
	#match template to image
	res = cv2.matchTemplate(image_binary,template_binary,method)
	(yCoords, xCoords) = np.where(res >= RES_THRESHOLD_BLACK)


	# initialize our list of rectangles
	rects = []
	# loop over the starting (x, y)-coordinates again
	for (x, y) in zip(xCoords, yCoords):
		# update our list of rectangles
		rects.append((x, y, x + w, y + h))
	
	# apply non-maxima suppression to the rectangles
	pick = non_max_suppression_fast(np.array(rects), OVERLAP_THRESHOLD)
	#print("[INFO] {} matched locations after NMS".format(len(pick)))
	
	#create a list of centers of the rectangles
	cross_points = []
	for (startX, startY, endX, endY) in pick:
	    # draw the bounding box on the image
		cv2.rectangle(image_binary, (startX, startY), (endX, endY), (0, 0, 255), 3)
		cross_points.append([int((startX+endX)/2), int((startY+ endY)/2)]) 

	#cv2.imshow("binary of black", image_binary)
	#cv2.imshow("template", template_binary)

	if (len(pick) == 4) :
		is_found = True
	
	cross_points = np.array(cross_points)
	cv2.imwrite("template_cross_binary.png", image_binary)
	
	return is_found, cross_points


In [19]:
def warp_image(image, template, method, old_corners):
	"""Transforms the camera image to a map bird's eye view with only the map"""
	is_found, cross_points = find_4_corners(image, template, method)
	
	if is_found:
		ordered_crosses = order_points(cross_points)
		#store corners positions in case we have a frame where we don't see them anymore
		old_corners = ordered_crosses
	else : 
		#will become the last found corners, 
		ordered_crosses = old_corners

	ordered_crosses = np.float32(ordered_crosses)
	bird_corners = np.float32(BIRD_CORNERS)

	matrix = cv2.getPerspectiveTransform(ordered_crosses, bird_corners)
	bird_image = cv2.warpPerspective(image, matrix, (BIRD_WIDTH, BIRD_HEIGHT))

	return is_found, bird_image, old_corners

In [20]:
def create_obstacles(hsv_image):
	"""Finds the obstacles (orange parts) on the map and put them on the map"""
	#filter the orange color
	hsv_min, hsv_max = band_pass_hsv(BLUE_BGR)
	mask = cv2.inRange(hsv_image, HSV_BLUE_MIN, HSV_BLUE_MAX)
	output = cv2.bitwise_and(hsv_image,hsv_image, mask= mask)
	#cv2.imshow("output of blue filter", output)
	#convert map to binary (0 = no obstacle, 1 = obstacle)
	output_gray = output[:, :, 2]
	
	threshold = BINARY_THRESHOLD_BLUE
	ret, map = cv2.threshold(output_gray, threshold, 255, cv2.THRESH_BINARY)

	#map[map==255] = OBSTACLE_MAP_VALUE
	return map

In [21]:
def dilate_obstacles(map, width):
	"""Dilates the obstacles to approximate the size of the thymio to one pixel
	map : [array] of 0 (background) and 1 (obstacles)
	width : [int] half the width of the thymio in pixels"""

	dilation_shape = cv2.MORPH_RECT
	kernel = cv2.getStructuringElement(dilation_shape, (2*width + 1, 2*width + 1), (width, width))
	eroded_image = cv2.erode(map, kernel, iterations = 1)
	dilated_img = cv2.dilate(eroded_image, kernel, iterations = 2)

	return dilated_img

In [22]:
def create_map_borders(map, width):
	"""put the borders of the map as obstacles"""
	map[0:width, :MAP_WIDTH] = 255
	map[:MAP_HEIGHT, MAP_WIDTH-width:MAP_WIDTH] = 255
	map[MAP_HEIGHT-width: MAP_HEIGHT, :MAP_WIDTH] = 255
	map[:MAP_HEIGHT, 0:width] = 255
	return map

In [23]:
def create_map(hsv_image):
	"""take the bird's eye image and convert it to a map, without the goal coordinates"""
	bird_corners = np.array(BIRD_CORNERS, dtype = np.float32)
	map_corners = np.array(MAP_CORNERS, dtype = np.float32)
	matrix = cv2.getPerspectiveTransform(bird_corners, map_corners)
	bird_image = cv2.warpPerspective(hsv_image, matrix, (MAP_WIDTH, MAP_HEIGHT))
	
	obstacles = create_obstacles(bird_image)
	
	
	
	dilated_map = dilate_obstacles(obstacles, THYMIO_WIDTH)
	cv2.imwrite("map_inside_createMap.jpg", dilated_map)
	map = create_map_borders(dilated_map, THYMIO_WIDTH)
	#map[map == 255] = OBSTACLE_MAP_VALUE
	return map

In [24]:
def get_goal_coord(hsv_img, template, method):
	"""Finds the position of the goal in the map, returns corners of rectangle and position
	| hsv_img : image in the hsv colorspace
	| template : template in binary colorspace of the goal
	| method : the cross correlation method (see exercises week 3)"""
	#keep only the green (color of goal)
	#bw = filter_hsv_to_binary(hsv_img, GREEN_BGR, BINARY_THRESHOLD_GREEN)
	#cv2.imshow("hsv img", hsv_img)
	#hsv_min, hsv_max = band_pass_hsv(bgr_color)
	mask = cv2.inRange(hsv_img, HSV_GREEN_MIN, HSV_GREEN_MAX)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)
	#cv2.imshow("green banddpass", output)
	grayscale = output[:, :, 2]
	#convert to binary
	threshold = BINARY_THRESHOLD_GREEN
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
	
	template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
	ret, template_bw = cv2.threshold(template_gray, BINARY_THRESHOLD_GREEN, 255, cv2.THRESH_BINARY)

	#cv2.imshow("green filter", bw)

	#find template coordinates
	res = cv2.matchTemplate(bw, template_bw, method)
	min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
	#get template dimensions
	c, w, h  = template.shape[::-1]
	#determine if we have found the coordinates or if it might be a fake
	if max_val >= RES_THRESHOLD_RED:
		is_found = True
		top_left = max_loc
		bottom_right = (top_left[0] + w, top_left[1] + h)
		position = (top_left[0] + int(w/2), top_left[1] + int(h/2))
	else:
		is_found = False
		top_left = (0, 0)
		bottom_right = (w, h)#set the thymio position to the top left corner if we haven't found it
		position = (0, 0)

	cX, cY = position
	position = (cY, cX) #invert positions to make the axis like those of a np array
	return is_found, position


In [25]:
def find_goal(hsv_img):
	"""Finds coordinates of thymio
	arguments :	
	| bw : image in the binary color space;	
	| template : image that has been filtered and is in the binary color space (1 dim);	
	| method : cross-correlation method for template matching
	"""
	is_found = False
	mask = cv2.inRange(hsv_img, HSV_GREEN_MIN, HSV_GREEN_MAX)
	output = cv2.bitwise_and(hsv_img,hsv_img, mask= mask)
	#cv2.imshow("green banddpass", output)
	grayscale = output[:, :, 2]
	#convert to binary
	threshold = BINARY_THRESHOLD_GREEN
	ret, bw = cv2.threshold(grayscale, threshold, 255, cv2.THRESH_BINARY)
		
	#cv2.imshow("green filter", bw)
	#cv2.imwrite("red_filter.jpg", bw)
	
	#find location of thymio
	contours, hierarchy  = cv2.findContours(bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

	#print(len(contours), "elements found")
	if len(contours) != 0:
		#if contour detected, keep the biggest one (most likely the thymio)
		contours = sorted(contours, key = cv2.contourArea, reverse = True)
		contour = contours[0]

		contour_threshold = 30 #detect if contour found is too small
		M = cv2.moments(contour)
		if (M["m00"] != 0) and (cv2.contourArea(contour) >= contour_threshold) :
			cX = int(M["m10"]/M["m00"])
			cY = int(M["m01"]/M["m00"])
			is_found = True
			position = (cY//2, cX//2)
		else :
			#division by zero or no contour big enough found
			#print(M["m00"], "contour too small") 
			position = (0, 0)
			is_found = False
	else:
		#no contour found
		#print("no contours found")
		position = (0, 0)
		is_found = False
	
	return is_found, position

In [26]:
def find_l_thymio(bw):
	"""Finds coordinates of thymio
	arguments :	
	| bw : image in the binary color space;	
	| template : image that has been filtered and is in the binary color space (1 dim);	
	| method : cross-correlation method for template matching
	"""
	is_found = False

	#find location of thymio
	contours, hierarchy  = cv2.findContours(bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

	#print(len(contours), "elements found")
	if len(contours) >= 2:
		#if contour detected, keep the biggest one (most likely the thymio)
		contours = sorted(contours, key = cv2.contourArea, reverse = True)
		big_circle = contours[0]
		small_circle = contours[1]

		contour_threshold = 10 #detect if contour found is too small
		M_big = cv2.moments(big_circle)
		M_small = cv2.moments(small_circle)
		if (M_big["m00"] != 0) and (M_small["m00"] != 0) and (cv2.contourArea(big_circle) >= contour_threshold) and (cv2.contourArea(small_circle) >= contour_threshold):
			cX_big = int(M_big["m10"]/M_big["m00"])
			cY_big = int(M_big["m01"]/M_big["m00"])
			cX_small = int(M_small["m10"]/M_small["m00"])
			cY_small = int(M_small["m01"]/M_small["m00"])
			is_found = True
			position = ((cX_big + cX_small)//4, (cY_big + cY_small)//4)
			delta_x = cX_big - cX_small
			delta_y = cY_big - cY_small
			angle = -(np.arctan2(delta_y, delta_x))
		else :
			#division by zero or no contour big enough found
			#print(M["m00"], "contour too small") 
			position = (0, 0)
			is_found = False
			angle = 0
	else:
		#no contour found
		#print("no contours found")
		position = (0, 0)
		is_found = False
		angle = 0
		
	return is_found, position, angle

In [27]:
def find_thymio_angle(image, template, method, position):
	"""Finds the orientation of the thymio
	| image : binary image of the map, where we can do template matching
	| template : binary image of the sticker on top of the thymio (be careful of thymio's leds)
	| method : type of cross correlation (see exercise week 3)
	| position : position of the thymio"""
	w, h = template.shape[::-1]
	template_center = (TEMP_CENTER_X, TEMP_CENTER_Y)
	
	thymio_loc = image[position[1] - TEMP_CENTER_Y : position[1] + (h-TEMP_CENTER_Y), position[0] - TEMP_CENTER_X: position[0] + (w-TEMP_CENTER_X)]
	#for each angle, rotate template and compare it to what is in the picture.
	angle_scores = np.zeros(360)
	for i in range(360):
		rotation_mat = cv2.getRotationMatrix2D(template_center, i, 1.0)
		rotated = cv2.warpAffine(template, rotation_mat, (w, h))
		angle_scores[i] = cv2.matchTemplate(thymio_loc, rotated, method)[0][0]

	#the angle of the thymio is the one that has the highest correlation
	best_angle = np.argmax(angle_scores)
	value = angle_scores.max()
	#print("cross corr value", value)
	return best_angle, value

In [28]:
def is_camera_obstructed(image):
    img_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

	#convert to binary both image and template
    threshold = 150
    
    ret, image_binary = cv2.threshold(img_grayscale, threshold, 255, cv2.THRESH_BINARY)
    zero_points = cv2.findNonZero(image_binary)
    
    if (zero_points is None) or (len(zero_points) < 200):
        is_obstructed = True
    else:
        is_obstructed = False
    return is_obstructed

In [29]:
def get_thymio_pose(image, old_corners):
	"""returns position and angle of thymio, and a boolean that is true if thymio has been found
	| image : camera image in the bgr colorspace"""
	is_cam_obstructed = is_camera_obstructed(image)
	
	if not is_cam_obstructed:
		ordered_crosses = np.float32(old_corners)
		bird_corners = np.float32(BIRD_CORNERS)
		matrix = cv2.getPerspectiveTransform(ordered_crosses, bird_corners)
		bird_image = cv2.warpPerspective(image, matrix, (BIRD_WIDTH, BIRD_HEIGHT))
		hsv_img = filter_to_hsv(bird_image)
		#keep only red and filter to binary
		bw = filter_hsv_to_binary(hsv_img, RED_BGR, BINARY_THRESHOLD_RED) #i try to have the image in hsv this time
	
		cv2.imwrite("hsv_image_fed_to_red_filter.jpg", bw)
		cv2.imwrite("bird_image.jpg", bird_image)

		#find position of thymio
		is_found, position, angle = find_l_thymio(bw)
	else:
		is_found = False
		position = (0, 0)
		angle = 0

	
	position = position[::-1]
	# print("position_thymio", is_found, position, angle)
	print('............................................')
	
	return is_cam_obstructed, is_found, position, angle #for now, the position is a pixel location, will need to convert it later


In [30]:

def make_sure_corners_are_found(image, template, method):
	"""Function to calibrate the camera and make sure we can detect all the corners"""
	is_found, cross_points = find_4_corners(image, template, method)
	
	image_copy = image.copy()
	for point in cross_points:
		cv2.circle(image_copy, tuple(point), 20, (0, 0, 255), 3)
	return is_found

In [31]:
def add_goal_to_map(map, goal_position): #probably won't use it
	map[goal_position] = 2 #value to change for now
	return map

In [32]:
def add_start_to_map(map, position): #probably won't use it
	#might want to add a condition to be sure thymio is not on an obstacle 
	map[position] = 3 #value to change for now
	return map

In [33]:
def wait_for_cam_to_calibrate():
	while True:
		time.sleep(0.1)
		if cv2.waitKey(1) == ord('s'):
			break

In [34]:
def color_path(image, path):
	#fastest method computing time wise, but not the most visible
	image[path] = np.array([0, 0, 255])
	#way slower, but more visible
	#for point in path:
		#cv2.circle(image, point[::-1], 2, (0, 0, 255), -1)
	return image

In [35]:
def initialise_setup(cam):
	# Open the default camera
	
	#set it for later
	old_corners = CAMERA_CORNERS #variable that will be updated each time
	map = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
	bird_image = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
	hsv_img = np.zeros((BIRD_HEIGHT, BIRD_WIDTH, 3))
	i = 0
	while i !=30:
		i +=1
		ret, image = cam.read()
		
		is_thymio_found = False
		corners_found, bird_image, old_corners = warp_image(image, template_cross, method, old_corners)
		bird_image_copy = bird_image.copy() #picture
		
		print("corners found", corners_found)
		hsv_img = filter_to_hsv(bird_image)
		map = create_map(hsv_img)
		cv2.imwrite("map_inside_init.jpg", map)
		is_goal_found, goal_position = find_goal(hsv_img)
		try:
			is_cam_obstructed, is_thymio_found, thymio_position, angle = get_thymio_pose(image, old_corners)
		except:
			print("Crashed avoided")

		cv2.imshow("Bird's eye view", bird_image)

	# Release the capture and writer objects
	
	cv2.destroyAllWindows()
	map = np.float32(map)
	map[map == 255] = -1
	

	return map, goal_position, thymio_position, angle, old_corners

In [36]:
#set it for later
old_corners = CAMERA_CORNERS #variable that will be updated each time

is_cam_setup = False
is_cam_obstructed = False
map = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
bird_image = np.zeros((BIRD_HEIGHT, BIRD_WIDTH))
hsv_img = np.zeros((BIRD_HEIGHT, BIRD_WIDTH, 3))

### Path Planning Functions

In [37]:
def heuristic(a, b):
    # implement the manhattan distance heuristic
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [38]:
def path_planning_a_star(map_grid, start, goal):
    def lowest_f_costs(open_list, f_costs):
        # identify the node in open_list wwith the lowest f_cost
        lowest_idx = 0
        for i in range(len(open_list)):
            if f_costs[open_list[i]] < f_costs[open_list[lowest_idx]]:
                lowest_idx = i
        return lowest_idx
        
    # initialization
    open_list = [start] # track unexplored nodes
    explored = [] # track explored nodes
    
    came_from = {} # track parent node of each nodes
    g_costs = {start: 0} # g_cost at each nodes
    f_costs = {start: heuristic(start, goal)} # f_cost at each nodes = g_cost + heuristic
    
    operation_count = 0
    
    while open_list:
        # pop the node with the lowest f_cost from the open set
        current_idx = lowest_f_costs(open_list, f_costs)
        current_pos = open_list.pop(current_idx)
        explored.append(current_pos)

        # reconstruct path
        if current_pos == goal:
            break
        
        # get the neighbors of the current node
        neighbors = [
            (current_pos[0] - 1, current_pos[1] - 1), # top-left
            (current_pos[0] - 1, current_pos[1]),     # up
            (current_pos[0] - 1, current_pos[1] + 1), # top-right
            (current_pos[0], current_pos[1] - 1),     # left
            (current_pos[0], current_pos[1] + 1),     # right
            (current_pos[0] + 1, current_pos[1] - 1), # bottom-left
            (current_pos[0] + 1, current_pos[1]),     # down
            (current_pos[0] + 1, current_pos[1] + 1)  # bottom-right
        ]

        for neighbor in neighbors:
            # check if neighbor is within bounds and not an obstacle
            if (0 <= neighbor[0] < map_grid.shape[0]) and (0 <= neighbor[1] < map_grid.shape[1]):
                if map_grid[neighbor[0], neighbor[1]] != -1 and neighbor not in explored:
                    # calculate g_costs_updated, considering diagonal steps
                    # probably add cost to g_scores if cells are near obstacles
                    if (neighbor[0] != current_pos[0]) and (neighbor[1] != current_pos[1]):
                        g_costs_updated = g_costs[current_pos] + np.sqrt(2)  # diagonal move cost
                    else:
                        g_costs_updated = g_costs[current_pos] + 1  # ddjacent move cost

                    # record the best path
                    if neighbor not in g_costs or g_costs_updated < g_costs[neighbor]:
                        came_from[neighbor] = current_pos
                        g_costs[neighbor] = g_costs_updated
                        f_costs[neighbor] = g_costs_updated + heuristic(neighbor, goal)

                        if neighbor not in open_list:
                            open_list.append(neighbor)
                        
                        operation_count += 1

    # reconstruct path
    if current_pos == goal:
        path = []
        while current_pos in came_from:
            path.append(current_pos)
            current_pos = came_from[current_pos]
        path.append(start)
        path.reverse()
        return path # in (x, y)_v
    else:
        # no path was found
        return None

In [39]:
def thymio_transform_path(path, start):
    #transform the path to new reference with origin at starting point
    start_x, start_y = start
 
    path_transformed = []

    for x, y in path:

        new_x = (y - start_y)
        new_y = -(x - start_x)

        path_transformed.append((new_x, new_y))
        
    return path_transformed # in (x, y)_t

In [40]:
def path_combining(path):
    # initialize
    path_translate = path
    path_rotate = []
    path_combined = []
    
    # check if the map has at least 2 points
    if len(path) < 2:
        return [] 
    
    # calculate robot's orientation at each position
    for i in range(len(path_translate) - 1):
        dx = path_translate[i+1][0] - path_translate[i][0]
        dy = path_translate[i+1][1] - path_translate[i][1]
        
        angle = np.arctan2(dy, dx) # rad
        
        path_rotate.append(angle)
    
    # generate final path with both positions and orientation
    for i in range(len(path_translate) - 1):
        x_coordinate = path_translate[i][0]
        y_coordinate = path_translate[i][1]
        angle = path_rotate[i]
        
        path_combined.append((x_coordinate, y_coordinate, angle))
    
    if path:
        last_point = path[-1]
        last_angle = path_rotate[-1] if path_rotate else 0  # default to 0 if no angles calculated
        path_combined.append((last_point[0], last_point[1], last_angle))
    
    return path_combined # (x, y, theta_rad)_t 

In [41]:
def path_finalizing(path, start, scale_factor = SCALING_FACTOR):
    path = thymio_transform_path(path, start)
    path = path_combining(path)
    
    path_final = [path[0]]
    
    # simplify the path into critical points
    for pose in path[1:]:
        if pose[2] != path_final[-1][2]:
            path_final.append(pose)
    
    path_final.append(path[-1])

    path = [(x * scale_factor, y * scale_factor, z) for x, y, z in path]     
        
    return path_final # (x, y, theta_rad)_t_reduced_scaled

### Filtering Functions

The state variable for the Kalman filter are 

\begin{cases}
   x_{+} = x_{0} + v*\Delta_{t}*cos(\theta)\\
   y_{+} = y_{0} + v*\Delta_{t}*sin(\theta)\\
   \theta_{+} = \theta_{0} + \phi\\
   v_{right}\\
   v_{left}
\end{cases}

This allow us to first calculate the Jacobian matrix. Furthermore, we distinguish between two cases: the first case occurs when the camera is obscured, in which only the left and right speed states are accessible. The second case applies when the camera is not hidden, and all state variables are utilized.

To determine the components of the process noise and measurement noise matrices, we conducted several experiments with the Thymio. For example, we instructed it to travel a certain distance for 10 seconds before stopping. We then measured the distance it covered. By repeating this experiment multiple times, we obtained a dataset with varying positions, from which we extracted  the variance.


In [42]:
x_est= y_est =  0 
is_initialized = False

In [43]:

data_position = [1.85,1.83,1.86,1.87,1.84,1.87,1.85,1.84,1.83,1.85,1.84,1.83,1.85,1.87,1.85,1.86,1.84] #mm for 1step de 0.1s
data_angle = [2.24,2.21,2.20,2.18,2.185,2.17,2.19,2.165,2.14,2.12,2.185,2.15,2.16,2.15,2.12,2.13,2.16] #rad for 1 step every 0.1s
data_sp_based_pos = [18.5,18.3,18.6,18.7,18.4,18.7,18.5,18.4,18.3,18.5,18.7,18.5,18.6,18.4] #mm/s extracted from the position measured during 10s
data_mean_speed_measured = np.array([51.5,60,56,52.5,49.5,56,51.5,45.5,53,51.5,53.5,59.5,51.5,49,55,49.5])*THYMIO_MMS # mm/s, for the R matrix 
data_pos_odom =[2.91,2.93,2.97,2.92,2.93,2.91,2.90,2.91,2.97,2.90,2.90,2.90,2.87,2.89,2.90] #mm every 0.1s for 10s
data_angle_odom = [6.023,6.208,6.204,6.151,6.086,6.113,6.151,6.166,6.057,6.169,6.165,6.065,6.155,6.240,6.116]

#vision value
data_vision_position_x_y = [1.658,1.647,1.647, 1.647, 1.658, 1.6472, 1.636, 1.647, 1.636, 1.658, 1.647]
data_vision_angle = [6.248, 6.265, 6.248, 6.230, 6.248, 6.230, 6.248, 6.248, 6.248, 6.248]

# Calcul de la moyenne et de l'écart-type
#1
mean_pos = 0 #np.mean(data_position)
std_dev_pos = np.std(data_position)

#2
mean_angle = 0.3 #np.mean(data_angle)
std_dev_angle = np.std(data_angle)

#3
mean_spd = np.mean(data_mean_speed_measured)
std_dev_spd = np.std(data_mean_speed_measured)

#4
mean_spd_pos = np.mean(data_sp_based_pos)
std_dev_spd_pos = np.std(data_sp_based_pos)

#5
mean_pos_odom = np.mean(data_pos_odom)
std_pos_odom = np.std(data_pos_odom)

#6
mean_angle_odom = 0.30 #np.mean(data_angle_odom)
std_angle_odom = np.std(data_angle_odom)

#7
mean_vision_position_x_y = 0 #np.mean(data_vision_position_x_y)
std_vision_position_x_y = np.std(data_vision_position_x_y)

#8
mean_vision_angle = np.mean(data_vision_angle)
std_vision_angle = np.std(data_vision_angle)


x1 = np.linspace(mean_pos - 3*std_dev_pos, mean_pos + 3*std_dev_pos, 500)  # Intervalle autour de la moyenne
y1 = (1 / (std_dev_pos * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x1 - mean_pos) / std_dev_pos) ** 2)

x2 = np.linspace(mean_angle - 3*std_dev_angle, mean_angle + 3*std_dev_angle, 500)  # Intervalle autour de la moyenne
y2 = (1 / (std_dev_angle * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x2 - mean_angle) / std_dev_angle) ** 2)

x3 = np.linspace(mean_spd - 3*std_dev_spd, mean_spd + 3*std_dev_spd, 500)  # Intervalle autour de la moyenne
y3 = (1 / (std_dev_spd * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x3 - mean_spd) / std_dev_spd) ** 2)

x4 = np.linspace(mean_spd_pos - 3*std_dev_spd_pos , mean_spd_pos + 3*std_dev_spd_pos , 500)  # Intervalle autour de la moyenne
y4 = (1 / (std_dev_spd_pos  * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x4 - mean_spd_pos) / std_dev_spd_pos ) ** 2)

x5 = np.linspace(mean_pos_odom - 3*std_pos_odom , mean_pos_odom + 3*std_pos_odom , 500)  # Intervalle autour de la moyenne
y5 = (1 / (std_pos_odom  * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x5 - mean_pos_odom) / std_pos_odom ) ** 2)

x6 = np.linspace(mean_angle_odom - 3*std_angle_odom , mean_angle_odom + 3*std_angle_odom , 500)  # Intervalle autour de la moyenne
y6 = (1 / (std_angle_odom  * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x6 - mean_angle_odom) / std_angle_odom ) ** 2)

x7= np.linspace(mean_vision_position_x_y - 3*std_vision_position_x_y , mean_vision_position_x_y + 3*std_vision_position_x_y , 500)  # Intervalle autour de la moyenne
y7 = (1 / (std_vision_position_x_y  * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x7 - mean_vision_position_x_y) / std_vision_position_x_y ) ** 2)

x8= np.linspace(mean_vision_angle - 3*std_vision_angle , mean_vision_angle + 3*std_vision_angle , 500)  # Intervalle autour de la moyenne
y8 = (1 / (std_vision_angle  * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x8 - mean_vision_angle) / std_vision_angle ) ** 2)

#Valeur de la variance de la position 
variance_pos = np.var(data_position)

variance_angle = np.var(data_angle)

variance_speed = np.var(data_mean_speed_measured)

variance_speed_pos = np.var(data_sp_based_pos)

variance_pos_odom = np.var(data_pos_odom)

variance_angle_odom = np.var(data_angle_odom)

variance_position_vision = np.var(data_vision_position_x_y) 

variance_angle_vision = np.var(data_vision_angle)

print(f"La variance de la position est : {variance_pos}")#q_x and q_y
print(f"La variance de l'angle est : {variance_angle}")#q_theta
print(f"La variance de la vitesse basé sur la position est : {variance_speed_pos}")#q_v_x and q_v_y

print(f"La variance de la position basé sur l'odom est : {variance_pos_odom}") #R_x and R_y
print(f"La variance de la vitesse est : {variance_speed}")#R_v_x and R_v_y
print(f"La variance de l'angle basé sur l'odoom est : {variance_angle_odom}")#R_theta

print(f"La variance de la position basé sur la vision est : {variance_position_vision}")#R_x_vision
print(f"La variance de l'angle basé sur la vision est : {variance_angle_vision}")#R_angle_vision


La variance de la position est : 0.00017508650519031173
La variance de l'angle est : 0.0009884083044982715
La variance de la vitesse basé sur la position est : 0.017806122448979594
La variance de la position basé sur l'odom est : 0.0006906666666666697
La variance de la vitesse est : 1.3863517955859388
La variance de l'angle basé sur l'odoom est : 0.0034459288888888853
La variance de la position basé sur la vision est : 5.396694214876031e-05
La variance de l'angle basé sur la vision est : 9.008999999999663e-05


In [44]:
# Normalization
y1_normalized = y1 / max(y1)
y2_normalized = y2 / max(y2)
y3_normalized = y3 / max(y3)
y4_normalized = y4 / max(y4)
y5_normalized = y5 / max(y5)
y6_normalized = y6 / max(y6)
y7_normalized = y7 / max(y7)
y8_normalized = y8 / max(y8)

# Tracé des distributions sur le même graphique
plt.figure(figsize=(12, 8))
plt.xlim(-0.5, 0.5)

# Ajout de chaque courbe au graphique avec normalisation
plt.plot(x1, y1_normalized, label="Position (data_position)", alpha=0.8)
plt.plot(x2, y2_normalized, label="Angle (data_angle)", alpha=0.8)
# plt.plot(x3, y3_normalized, label="Vitesse mesurée (data_mean_speed_measured)", alpha=0.8)
# plt.plot(x4, y4_normalized, label="Vitesse basée sur la position (data_sp_based_pos)", alpha=0.8)
# plt.plot(x5, y5_normalized, label="Position (odométrie, data_pos_odom)", alpha=0.8)
plt.plot(x6, y6_normalized, label="Angle (odométrie, data_angle_odom)", alpha=0.8)
plt.plot(x7, y7_normalized, label="Position (vision, data_vision_position_x_y)", alpha=0.8)
# plt.plot(x8, y8_normalized, label="Angle (vision, data_vision_angle)", alpha=0.8)

# Configuration des axes et de la légende
plt.title("Distributions normalisées des données avec courbes gaussiennes", fontsize=16)
plt.xlabel("Valeurs normalisées", fontsize=14)
plt.ylabel("Densité de probabilité relative", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)

# Affichage du graphique
plt.show()

In [45]:
def odometrie(x_prev,y_prev,theta_prev,motor_l,motor_r):
   
    v = (motor_r + motor_l)*THYMIO_MMS/2
    w = (motor_r - motor_l)*THYMIO_MMS

    theta = theta_prev +  w*DELTA_T/D_BASELINE
    
    x = x_prev + v*DELTA_T*np.cos(theta_prev)
    y = y_prev + v*DELTA_T*np.sin(theta_prev)

    return x,y, theta , motor_r, motor_l

In [46]:
def Jacobien(theta, v_right, v_left): 
    """
    Computes the Jacobian matrix for the Kalman filter, 
    """
    alpha = THYMIO_MMS * (v_right + v_left) / 2
    beta = THYMIO_MMS * (v_right - v_left) / D_BASELINE

    A = - (alpha * DELTA_T * np.sin(theta + DELTA_T * beta))
    B = (1/2) * DELTA_T * np.cos(theta + DELTA_T * beta) - (alpha * (1 / D_BASELINE) * DELTA_T**2 * np.sin(theta + DELTA_T * beta))
    C = (1/2) * DELTA_T * np.cos(theta + DELTA_T * beta) + (alpha * (1 / D_BASELINE) * DELTA_T**2 * np.cos(theta + DELTA_T * beta))
    D = alpha * DELTA_T * np.cos(theta + DELTA_T * beta)
    E = (1/2) * DELTA_T * np.sin(theta + DELTA_T * beta) + (alpha * (1 / D_BASELINE) * DELTA_T**2 * np.cos(theta + DELTA_T * beta))
    F = (1/2) * DELTA_T * np.sin(theta + DELTA_T * beta) - (alpha * (1 / D_BASELINE) * DELTA_T**2 * np.cos(theta + DELTA_T * beta))
    G = DELTA_T/D_BASELINE
    H = -DELTA_T/D_BASELINE
    
    J = np.array([[1, 0, A, B, C],
                  [0, 1, D, E, F],
                  [0, 0, 1, G, H],
                  [0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 1]])
    
    return J

In [47]:

def kalman_filter(x_vision,y_vision,theta_vision,spd_right,spd_left,Sigma, is_cam_obstructed, theta_shift_vision_init):

        """
        Kalman Filter
        :param Q: Process noise matrix
        :param R: Measurement noise matrix
        """
        
        global Q,C_obstructed,C_not_obstructed,R_obstructed,R_not_obstructed, x_est, y_est, theta_est, is_initialized
        # print('valeur theta shift: ', theta_shift_vision_init)
        
        if not is_initialized:
                theta_est = theta_shift_vision_init
                # print('inside')
                is_initialized = True  

        q_x = q_y = 0.00017
        q_theta = 0.00098
        q_vx = q_vy = 0.0178

        r_x = r_y = 5.396e-05
        r_theta = 9.008e-05
        r_vx = r_vy = 1.213

        # #vision coeff
        # r_x_vision = r_y_vision = 5.396e-05
        # r_theta_vision = 9.008e-05
        # r_vx = r_vy = 1.213

        Q = np.array([
                [q_x,0,0,0,0],
                [0,q_y,0,0,0],
                [0,0,q_theta,0,0],
                [0,0,0,q_vx,0],
                [0,0,0,0,q_vy]
                ])

        C_obstructed = np.array([
                         [0,0,0,1,0],
                         [0,0,0,0,1],
                         ])
    
        C_not_obstructed = np.eye(5)
    
        R_obstructed = np.array([[1.21,0],
                            [0,1.21]])
    
        R_not_obstructed = np.array([
                                [r_x,0,0,0,0],
                                [0,r_y,0,0,0],
                                [0,0,r_theta,0,0],
                                [0,0,0, r_vx,0],
                                [0,0,0,0, r_vy]
                                ])


        # 1/ Estimation of mean of states
        theta_est = normalize_angle(theta_est)
        # print('x_odom, y_odom and theta_odom from inside the kalmann: ', x_est, y_est, np.rad2deg(theta_est))
        x_est, y_est, theta_est, spd_right_est, spd_left_est = odometrie(x_est,y_est,theta_est,spd_right,spd_left)
        # theta_est = normalize_angle(theta_est) #added 04.12.2024
        
        mu_est = np.array([[x_est],[y_est],[theta_est],[spd_right_est],[spd_left_est]])
        
        # 2/ Estimation of variance of states
        jacobien = Jacobien(theta_est,spd_right_est,spd_left_est)
        Sigma_est = np.matmul(np.matmul(jacobien,Sigma),np.transpose(jacobien))+Q
        
        # test if vision is obstructed or not:
        if is_cam_obstructed:
         
                # 3/ Gain de correction optimal
                S = np.matmul(np.matmul(C_obstructed,Sigma_est),np.transpose(C_obstructed))+R_obstructed
                K = np.matmul(np.matmul(Sigma_est,np.transpose(C_obstructed)),np.linalg.inv(S))
        
                # 4/ Update with the measure 
                y = np.array([[spd_right_est],[spd_left_est]])
                y_reel = np.array([[spd_right],[spd_left]]) 
                mu_updated = mu_est + np.matmul(K,np.subtract(y_reel,y))
        
                # 5/ Update the variance
                Sigma_updated = np.matmul(np.subtract(np.eye(5),np.matmul(K,C_obstructed)),Sigma_est)

                return mu_updated[0][0],mu_updated[1][0],mu_updated[2][0],mu_updated[3][0],mu_updated[4][0],Sigma_updated # mu_est for x,y and theta and mu_reel for speeds
        
        else: # cam is not obstructed
                
                # 3/ Gain de correction optimal
                
                S = np.matmul(np.matmul(C_not_obstructed,Sigma_est),np.transpose(C_not_obstructed))+R_not_obstructed
                K = np.matmul(np.matmul(Sigma_est,np.transpose(C_not_obstructed)),np.linalg.inv(S))
        
                # 4/ Update avec la mesure (moyenne)
                y = np.array([[x_est],[y_est],[theta_est],[spd_right_est],[spd_left_est]])
                # y = np.array([x_est, y_est, theta_est, spd_right_est, spd_left_est]).reshape((-1, 1))
                y_reel = np.array([[x_vision],[y_vision],[theta_vision],[spd_right],[spd_left]]) #=> Changez pour vos fonctions !!!
                # y_reel = np.array([x, y, theta, spd_right, spd_left]).reshape((-1, 1))
                mu_updated = mu_est + np.matmul(K,np.subtract(y_reel,y))
                # mu_updated = mu_est + np.matmul(K, y_reel - y)

                # 5/ Update de la variance avec la mesure
                Sigma_updated = np.matmul(np.subtract(np.eye(5),np.matmul(K,C_not_obstructed)),Sigma_est)
                
                return mu_updated[0][0],mu_updated[1][0],mu_updated[2][0],mu_updated[3][0],mu_updated[4][0],Sigma_updated # mu_reel for all

### Motion Control Functions

In [48]:
motor_speed = 100
count_control = 0

In [49]:
# def p_control(angle_error, kp, motor_speed):
#     p_correction = kp * abs(angle_error)

#     if angle_error < 0.0:
#         motor_left = int(motor_speed + p_correction)
#         motor_right = int(motor_speed - p_correction)
#     else:
#         motor_left = int(motor_speed - p_correction)
#         motor_right = int(motor_speed + p_correction)
    
#     return motor_left, motor_right

def p_control(angle_error, kp, motor_speed):
    if abs(angle_error) < np.pi:
        p_correction = kp * abs(angle_error) 
        if angle_error < 0.0:
           motor_left = int(motor_speed + p_correction)
           motor_right = int(motor_speed - p_correction)
        else:
           motor_left = int(motor_speed - p_correction)
           motor_right = int(motor_speed + p_correction)
    else:
        p_correction = kp * abs(2 * np.pi - angle_error)
        if angle_error > 0.0:
           motor_left = int(motor_speed + p_correction)
           motor_right = int(motor_speed - p_correction)
        else:
           motor_left = int(motor_speed - p_correction)
           motor_right = int(motor_speed + p_correction)
    
    return motor_left, motor_right 

In [50]:
count_control = 0
async def motion_control(pose, path):
    global count_control

    if count_control < len(path): # motion control 
        sub_goal = path[count_control]
        
        dist_x = sub_goal[0] - pose[0] # distance in x-direction
        dist_y = sub_goal[1] - pose[1] # distance in y-direction
        
        angle_target = np.arctan2(dist_y, dist_x) # in (x, y, theta_rad)_t
        # angle_robot = pose[2]
        angle_robot = normalize_angle(pose[2]) # theta_filtered
        # angle_error = normalize_angle(angle_target - angle_robot)
        angle_error = angle_target - angle_robot
        
        dist_to_goal = np.sqrt((dist_x)**2 + dist_y**2) # distance from current pose to subgoal
        

        if sub_goal[0] == sub_goal[1] == 0 and abs(sub_goal[2] - angle_robot) > np.deg2rad(15):
            left_speed, right_speed = p_control(sub_goal[2] - angle_robot, 25, motor_speed = 0)
            await node.set_variables(motors(left_speed, right_speed))
            print('left: ', left_speed, 'right: ', right_speed)
            print('starting: ', np.rad2deg(sub_goal[2]) - np.rad2deg(angle_robot), np.rad2deg(sub_goal[2]), np.rad2deg(angle_robot))

        
        elif dist_to_goal >= 15: # SUCCESS_THRESHOLD
            # if abs(angle_error) > np.deg2rad(20):
            #     left_speed, right_speed = p_control(angle_error, 20, motor_speed = 0)
            #     await node.set_variables(motors(left_speed, right_speed)) test

            if dist_to_goal <= 70:
                left_speed, right_speed = p_control(angle_error, 15, 40)
                await node.set_variables(motors(left_speed, right_speed))
                print('pose_x_fil: ', pose[0], 'pose_y_fil: ', pose[1])
                print('dist small', dist_to_goal)
                print('to goal angle: ', np.rad2deg(angle_target))
                print('angle_fil: ', np.rad2deg(angle_robot))
                print('angle difference: ', np.rad2deg(angle_target - angle_robot))
                #print('angle error: ', np.rad2deg(angle_error))
                print('left: ', left_speed, 'right: ', right_speed)
                
            else:
                left_speed, right_speed = p_control(angle_error, 25, 75)
                await node.set_variables(motors(left_speed, right_speed))
                print('pose_x_fil: ', pose[0], 'pose_y_fil: ', pose[1])
                print('dist big', dist_to_goal) 
                print('to goal angle: ', np.rad2deg(angle_target))
                print('angle_fil: ', np.rad2deg(angle_robot))
                print('angle difference: ', np.rad2deg(angle_target - angle_robot))
                #print('angle error: ', np.rad2deg(angle_error))
                print('left: ', left_speed, 'right: ', right_speed)   
            
        else:
            # print('goal reached')
            # if abs(sub_goal[2] - angle_robot) > np.deg2rad(35):
            #     left_speed, right_speed = p_control(sub_goal[2] - angle_robot, 30, motor_speed = 0) #if abs(left_speed) > 15 else 15
            #     await node.set_variables(motors(left_speed, right_speed))
            #     print('rotating')
            #     print('to goal angle: ', np.rad2deg(angle_target))
            #     print('angle_fil: ', np.rad2deg(angle_robot))
            #     print('angle difference: ', np.rad2deg(angle_target - angle_robot))
            #     #print('angle error: ', np.rad2deg(angle_error))
            #     print('left: ', left_speed, 'right: ', right_speed)   
            # else: 
            print('correction finished')
            count_control += 1
        print(sub_goal)
        print('count: ', count_control)

    else: 
        await node.set_variables(motors(0,0))

### Main

In [51]:
# Filtering Part
x_odom_array = []
y_odom_array = []


sigma = np.zeros(5)
x_kalman = []
y_kalman = []

#Initialisation des graphiques
fig, ax, line = initialize_plot()

In [52]:
async def main():
    global count_control, x_filtered, y_filtered, theta_filtered, x_odom, y_odom, theta_odom, sigma, is_cam_obstructed, thymio_position_vision,is_thymio_found,thymio_angle_vision, path, search_goal, search_start, map 

    x_odom = y_odom = 0

    # VISION
    # open the default camera
    cam = cv2.VideoCapture(1)
    if not cam.isOpened():
            print("Error: Could not open camera.")

    # convert camera resolution from 1920x1080 to 1024x768
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 768)

    map, search_goal, search_start, theta_vision_init, old_corners = initialise_setup(cam) # in (x, y, theta_rad)_v
    
    map_image = np.float32(map.copy())
    map_image[map == -1] = 255
    cv2.imwrite("blue filter.jpg", map_image)
    
    # PATH PLANNING
    print(search_goal, search_start)
    print('hahah')
    print(map)
    path = path_planning_a_star(map, search_start, search_goal) 
    path = path_finalizing(path, search_start, SCALING_FACTOR) # in (x, y, theta_rad)_t
    path = [(float(x), float(y), float(theta)) for x, y, theta in path]
    path_print = [(float(x), float(y), float(np.rad2deg(theta))) for x, y, theta in path]
    print('path: ', path_print)

    while True:
        start_time = time.time()
        ret, image = cam.read()
        await node.wait_for_variables({
            "motor.left.speed",
            "motor.right.speed",
            "button.center",
            "button.forward",
            "prox.horizontal",
            "acc"
        })

        # variables
        motor_left = node["motor.left.speed"]
        motor_right = node["motor.right.speed"]
        button_center = node["button.center"]
        button_forward = node["button.forward"]
        prox_horizontal = node["prox.horizontal"] #tableau contenant les 7 capteurs de proximité
        acceleration = node["acc"]
        
        # FILTERING
        is_cam_obstructed, is_thymio_found, thymio_position_vision, theta_vision = get_thymio_pose(image, old_corners)
        x_vision, y_vision = thymio_transform_pose(thymio_position_vision, search_start) # in (x, y, theta_rad)_t
        print('pose_x_vi: ', x_vision, 'pose_y_vi: ', y_vision)
        print('angle_vi: ', np.rad2deg(theta_vision))

        x_filtered, y_filtered,theta_filtered, _, _, sigma = kalman_filter(x_vision, y_vision, theta_vision, motor_right, motor_left, sigma, is_cam_obstructed, theta_vision_init) # in (x, y, theta_rad)_t
        # theta_filtered = normalize_angle(theta_filtered) # added 04.12.2024
        
        print('angle filtered', np.rad2deg(theta_filtered))
        # MOTION CONTROL
        pose = (float(x_filtered), float(y_filtered), float(theta_vision))
        # pose = (float(x_filtered), float(y_filtered), float(theta_filtered)) #filtered not working
        await motion_control(pose, path)

        # PLOTTING
        x_kalman.append(x_filtered)
        y_kalman.append(y_filtered)
        line.set_data(x_kalman,y_kalman)

        ## erase previous ellipses
        # for patch in reversed(ax.patches):
        #     patch.remove()

        # add ellipses
        # plot_gaussian(ax, [x_odom, y_odom], Sigma[:2, :2], color='red') #on prend uniquement la partie de la matrice de variance correspondante à x et y donc une sub matrice 2x2
        fig.canvas.draw()

        fig.canvas.flush_events()

        # THYMIO DIRECT CONTROL
        if button_forward == 1:
            await node.set_variables(motors(100, 100))

        if button_center == 1:
            await node.set_variables(motors(0, 0))
            x_odom = y_odom = theta_odom = 0
            break

    cam.release()
    cv2.destroyAllWindows()

    time_diff = time.time() - start_time

    if time_diff < DELTA_T:
        await client.sleep(DELTA_T - time_diff)
    
await main()

corners found True
............................................
corners found True
............................................
corners found True
............................................
corners found True
............................................
corners found True
............................................
corners found False
............................................
corners found True
............................................
corners found True
............................................
corners found False
............................................
corners found False
............................................
corners found False
............................................
corners found False
............................................
corners found False
............................................
corners found False
............................................
corners found False
............................................
corners found False
............

In [54]:
path = path_planning_a_star(map, search_start, search_goal) 
path_show = thymio_transform_path(path, search_start)

# path = path_finalizing(path, search_start, SCALING_FACTOR) # in (x, y, theta_rad)_t
# path = [(float(x), float(y), float(theta)) for x, y, theta in path]
display_map(map, path, search_start, search_goal)
print(path_show)
print(search_start, search_goal)

[(0, 0), (-1, 1), (-2, 2), (-3, 3), (-4, 4), (-5, 5), (-6, 6), (-7, 7), (-8, 8), (-9, 9), (-10, 10), (-11, 11), (-12, 12), (-13, 13), (-14, 14), (-15, 15), (-16, 16), (-17, 17), (-18, 18), (-19, 19), (-20, 20), (-21, 21), (-22, 22), (-23, 23), (-24, 24), (-25, 25), (-26, 26), (-27, 27), (-28, 28), (-29, 29), (-30, 30), (-31, 31), (-32, 32), (-33, 33), (-34, 34), (-35, 35), (-36, 36), (-37, 37), (-38, 38), (-39, 39), (-40, 40), (-41, 41), (-42, 42), (-43, 43), (-44, 44), (-45, 45), (-46, 46), (-47, 47), (-48, 48), (-49, 49), (-50, 50), (-51, 51), (-52, 52), (-53, 53), (-54, 54), (-55, 55), (-56, 56), (-57, 57), (-58, 58), (-59, 58), (-60, 58), (-61, 58), (-62, 58), (-63, 58), (-64, 58), (-65, 58), (-66, 58), (-67, 58), (-68, 58), (-69, 58), (-70, 58), (-71, 58), (-72, 58), (-73, 58), (-74, 58), (-75, 58), (-76, 58), (-77, 58), (-78, 58), (-79, 58), (-80, 58), (-81, 58), (-82, 58), (-83, 58), (-84, 58), (-85, 58), (-86, 58), (-87, 58), (-88, 58), (-89, 58), (-90, 58), (-91, 58), (-92, 58

Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 73, in run
    packet = self.read_packet()
             ^^^^^^^^^^^^^^^^^^
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 66, in read_packet
    raise error
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 59, in read_packet
    packet_len = self.read_uint32()
                 ^^^^^^^^^^^^^^^^^^
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 46, in read_uint32
    b = self.io.read(4)
        ^^^^^^^^^^^^^^^
  File "c:\Users\tjga9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tdmclient\tcp.py", line 99, in read
  